# 3.5 Accounting for error in simple models

In this notebook, we explore some of the ways that error can enter into a model, and methods for handling it. The major forms of error and uncertainty are:
1. Observational error. 
2. Parameter uncertainty.
3. Prediction uncertainty.
4. Structural error.

Each is demonstrated by it's own code snippet below for the case of a simple linear model with two parameters: gradient, $m_0$, and $y$-intercept, $c_0$. The model is denoted 

$$y = mx+c = f(x;\boldsymbol{\theta}),$$ 

where $\boldsymbol{\theta} = [m,c]$ is the parameter vector, and $\boldsymbol{\theta}_0$ its true value we are trying to determine.

## 3.5.1 Observational error

We concede that no instrument makes perfect measurements, and therefore there is some amount of random error, $\epsilon$, in an observation, i.e., $ \tilde{y}_i = y_i+\epsilon$, where $y_i$ is the true value of a quantity we are trying to measure, and $\tilde{y_i}$ is the actual measurement made.

We assume that $\epsilon$ is a normally distributed random variable with variance, $\sigma_i^2$.

For the simple linear model, we demonstrate the "observation" process by sampling the true model, $f(x;\boldsymbol{\theta}_0)$, at the observation points, $x_i$, and adding normally distributed random error. 

Execute the code below to see how the observation points (open circles) deviate from the true model (blue line). You can also plot the best-fit model.

***Is the best-fit model the same as the "true" model?***

In [ ]:
%matplotlib inline

# import modules
from scipy.stats import multivariate_normal
from scipy.optimize import curve_fit
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from ipywidgets import interact 

# define a model
ts = 14.
x = np.linspace(0,1,101)

# model parameters
m0 = 2.       # true gradient
c0 = 3.       # true intercept
v = 0.15      # variance of random error when making measurements

# define the model, a simple linear function
def func(x,m,c): 
    return m*x+c

# define the error, a random draw from a normal distribution
def e(x,v): 
    return np.random.randn(len(x))*np.sqrt(v)

# compute the "true" model, using the "true" parameters
y = func(x,m0,c0)

# seed the random number generator so we get the same numbers each time
np.random.seed(13)

# define some values of the independent variable at which we will be taking our "observations"
xo = np.linspace(0,1,12)[1:-1]

# compute the observations - "true" model + random error (drawn from normal distribution)
yo = func(xo,m0,c0) + e(xo,v)

def plot_observations(N_obs, true_model, RMS_fit, error_dist):
        # check N_obs does not exceed length of observations
    i = np.min([len(xo), N_obs])
        # initialize figure window and axes
    fig, ax = plt.subplots(nrows=1, ncols=1)
    fig.set_size_inches([8.,5.])
        # plot the "true" model    
    if true_model:
        ln1 = ax.plot(x,y,'b-', label = 'true model',zorder = 10)
    else:
        ln1 = []
        # plot the observations
    ln2 = ax.plot(xo[:i],yo[:i],'wo', mec = 'k', mew = 1.5, ms = 5, label = r'observations', zorder = 10)
        # add "best-fit" model if appropriate
    if RMS_fit:
        # find best-fit model
        p2,pc = curve_fit(func, xo[:i], yo[:i], [1,1])
        # plot model
        ax.plot(x,func(x,*p2),'r-')
    
        # add normal distributions
    ylim = ax.get_ylim()
    ye = np.linspace(-ylim[1],ylim[1],101)*0.2
    ye2 = np.linspace(-ylim[1],ylim[1],101)*0.25
        # loop over plotted observations
    for xoi, yoi in zip(xo[:i],yo[:i]):
            # normal dist
        xi = 0.05*np.exp(-(ye)**2/v)+xoi
            # add to plot
        if error_dist:
            ax.plot(xi, ye+func(xoi,m0,c0), 'k-', lw = 0.5, zorder = 0)
            ax.plot(xi*.0+xoi, ye2+func(xoi,m0,c0), '-', lw = 0.5, zorder = 0, color = [0.5, 0.5, 0.5])

        # plot upkeep + legend
    ax.set_xlim(ax.get_xlim())
    lns = ln1+ln2
    lbs = [ln.get_label() for ln in lns]
    ax.legend(lns,lbs,loc = 2,prop={'size':ts})
    ax.set_ylim([1,7])
    ax.set_xlim([0,1])
    ax.set_xlabel('x',size = ts)
    ax.set_ylabel('y',size = ts)
    for t in ax.get_xticklabels()+ax.get_yticklabels(): t.set_fontsize(ts)
    plt.show()
    
interact(plot_observations, N_obs = (2,10,1), true_model = True, RMS_fit = False, error_dist = False)

## 3.5.2 Parameter uncertainty

### 3.5.2.1 Grid search parameter estimation

We use observations to learn about parameters during model calibration. However, if the observations are uncertain, then so too must be the parameters.

One way to describe parameter uncertainty is via a *probability distribution*. When that distribution has been informed by the observations, we call it the *posterior* and denote it $P(\boldsymbol{\theta}|\tilde{y}_i)$.

For a uniform *prior distribution* (our initial expectations about the parameter, before seeing any data), and normally distributed errors, we can compute the posterior directly.

$$ r(\boldsymbol{\theta}) = -\sum_i^n \frac{1}{\sigma_i^2}(\tilde{y}_i-f(x_i;\boldsymbol{\theta}))^2, \quad\quad\quad\quad P(\boldsymbol{\theta}|\tilde{y}_i) = A \exp(r/2),$$

where $r$ is the objective function to be maximized during calibration, and $A$ is a normalizing constant.

We compute $r(\boldsymbol{\theta})$ by performing a grid search over parameter space, running the model and computing the objective function at discrete points, $\boldsymbol{\theta}_i$.

Execute the code below to see a plot of the posterior over parameter space. Use the slider bars to manually adjust the parameters of an arbitrary model, and see how the fit of this model to the data (left plot) corresponds to different likelihoods of the posterior (right).

***Does the true model correspond to the maximum of the posterior? Explain.***

In [ ]:
# generate parameter grid for grid search
m = np.linspace(m0-2,m0+2,31); dm = m[1]-m[0]
c = np.linspace(c0-1,c0+1,31); dc = c[1]-c[0]
M,C = np.meshgrid(m,c)

# compute objective function
    # empty vector, correct size, for storing computed objective function
r = 0.*M.flatten() 
    # for each parameter combination in the grid search
for i,theta in enumerate(zip(M.flatten(), C.flatten())):
        # unpack parameter vector
    mi,ci = theta
        # compute objective function
    r[i]=-np.sum((yo-func(xo,mi,ci))**2)/v
    # reshape objective function to meshgrid dimensions
R = np.array(r).reshape([len(c), len(m)])
    # compute posterior
post = np.exp(R/2.)
    # convert 2D mesh to vectors
mv, cv, pv = [vi.flatten() for vi in [M,C,post]]
    
# plotting function
def plot_posterior(m1,c1):
        # initialize figure window and axes
    fig = plt.figure(figsize=[16.,6.5])
    ax1 = plt.axes([0.15,0.15,0.35,0.75])
    ax2 = fig.add_subplot(122, projection='3d')
    
    # show data and fitted models
        # plot the "true" model    
    ln1 = ax1.plot(x,y,'b-', label = 'true model',zorder = 10)
        # plot the observations
    ln2 = ax1.plot(xo,yo,'wo', mec = 'k', mew = 1.5, ms = 5, label = r'observations', zorder = 10)
        # best-fit model
    p2,pc = curve_fit(func, xo, yo, [1,1])
    ln3 = ax1.plot(x,func(x,*p2),'r-', label = 'best-fit model')
        # show model with [m1,c1]
    i,j = np.argmin(abs(m-m1)), np.argmin(abs(c-c1))    # snap to nearest grid value
    ln4 = ax1.plot(x,func(x,m[i],c[j]),'g-', label = 'aribtrary model: r=%3.2f'%R[i,j])
    lns = ln1+ln2+ln3+ln4
    lbs = [ln.get_label() for ln in lns]
    ax1.legend(lns,lbs,loc=2)
    
    # show posterior
        # plot posterior as surface over 2D parameter space
    ax2.plot_surface(M, C, post, rstride=1, cstride=1,cmap=cm.Oranges, lw = 0.5, zorder = 10)
    
    # show models on posterior
        # best-fit model
    im = np.argmax(pv)
    ax2.plot3D([mv[im],],[cv[im],],[pv[im],],'ro',mec = 'r', ms = 7)
        # true model
    ax2.plot3D([m0,],[c0,],[np.exp(-np.sum((yo-func(xo,m0,c0))**2)/(2.*v)),],'bo',mec = 'b', ms = 7)
        # arbitrary model
    im = np.argmax(pv)
    ax2.plot3D([M[j,i],],[C[j,i],],[post[j,i],],'go',mec = 'g', ms = 7)

    # plot upkeep, labels
    ax2.set_xlabel('m',size = ts)
    ax2.set_ylabel('c',size = ts)
    ax2.set_xticklabels([])
    ax2.set_yticklabels([])
    ax2.set_zticklabels([])
    ax1.set_xlabel('x',size = ts)
    ax1.set_ylabel('y',size = ts)
    for ax in [ax1,ax2]:
        for t in ax.get_xticklabels()+ax.get_yticklabels(): t.set_fontsize(ts)
        
    # plot view angle
    ax2.view_init(45, 15-90)
    plt.show()
    
interact(plot_posterior, m1 = (m0-2,m0+2,4./51), c1 = (c0-1,c0+1,2./51))

### 3.5.2.2 Monte-Carlo parameter estimation

Another method for estimating posterior parameter distributions is Markov-Chain Monte-Carlo. The mathematics of the method are well beyond the scope of this notebook (but there are plenty of excellent descriptions online). 

There is a excellent Python package called [emcee](http://dan.iel.fm/emcee/current/) that implements a number of MCMC algorithms. We will use the package to replicate the grid search above.

If you are using the Anaconda Python distribution, then emcee can be installed from the command line using `pip`

`pip install emcee`

In [ ]:
import emcee

# log likelihood for the model, given the data
def lnprob(pars, obs):
    v = 0.15
    return -np.sum((obs[:,1]-func(obs[:,0],*pars))**2)/v
    
ndim = 2                # parameter space dimensionality
nwalkers=10             # number of walkers

# create the emcee object (set threads>1 for multiprocessing)
data = np.array([xo,yo]).T
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, threads=1, args=[data,])

# set the initial location of the walkers
pars = [2.5, 2.5]       # initial guess
p0 = np.array([pars + 1e-3*np.random.randn(ndim) for i in range(nwalkers)])  # add some noise

# set the emcee sampler to start at the initial guess and run 100 burn-in jumps
pos,prob,state=sampler.run_mcmc(p0,100)
sampler.reset()

# run 1000 jumps and save the results 
pos,prob,state=sampler.run_mcmc(pos,1000)
f = open("chain.dat", "w")
nk,nit,ndim=sampler.chain.shape
for k in range(nk):
    for i in range(nit):
        f.write("{:d} {:d} ".format(k, i))
        for j in range(ndim):
            f.write("{:15.7f} ".format(sampler.chain[k,i,j]))
        f.write("{:15.7f}\n".format(sampler.lnprobability[k,i]))
f.close()

We'll use the corner module to display some of the results. This can be installed by `pip` in the same way as `emcee`.

In [ ]:
# show corner plot of the results
import corner
chain = np.genfromtxt('chain.dat')
weights = chain[:,-1]
weights -= np.max(weights)
weights = np.exp(weights)
labels = ['m','c']
fig = corner.corner(chain[:,2:-1], labels=labels, weights=weights, smooth=1, bins=30)

## 3.5.3 Prediction uncertainty

Establishing the posterior gives us a quantitative measure of which parameter combinations are more likely than others. To make a forecast of the future (a probabilistic description of all outcomes), we sample parameter combinations from the posterior and use the model to make predictions for these. The key is that predictions corresponding to the more likely parameter combinations will have a greater weight in the final forecast.

Execute the code below. Use the slider bar to take more and more samples from the posterior. Note how the forecast - a histogram of predictions - starts to approximate a probability distribution as more and more samples are drawn. 

***How well does the 90% forecast interval approximate the true model?***

In [ ]:
# construct covariance matrix
    # means
pv = pv/np.sum(pv)
m1 = np.sum(pv*mv)
c1 = np.sum(pv*cv)
    # variances
smm = np.sum(pv*(mv-m1)**2)
scc = np.sum(pv*(cv-c1)**2)
scm = np.sum(pv*(mv-m1)*(cv-c1))
    # matrix
cov = np.array([[smm,scm],[scm,scc]])

# plotting function
def plot_ensemble(logN,predict):
        # initialize figure window and axes
    fig, axs = plt.subplots(nrows=1, ncols=2)
    fig.set_size_inches([16.,5.])
    
        # vector for long range forecast
    if predict:
        x2 = np.linspace(0,3.2, 101)
        ix = np.argmin(abs(x2 - 3.0))
    else:
        x2 = np.linspace(0,1,101)

        # plot the "true" model  
    ln1 = axs[0].plot(x2,func(x2,m0,c0),'b-', label = 'true model',zorder = 10)
        # plot the observations
    ln2 = axs[0].plot(xo,yo,'wo', mec = 'k', mew = 1.5, ms = 5, label = r'observations', zorder = 10)
    
        # take samples from multivariate normal
    np.random.seed(13)
    samples = multivariate_normal.rvs(mean = [m1, c1], cov = cov, size = 2**logN)
    if logN == 0: samples = [samples,]
        # plot line for each sample      
    prediction = []
    for i,s in enumerate(samples):
            # plot sample model
        if i==0:
            ln3 = axs[0].plot(x2,func(x2,*s),'k-', zorder = 0, lw = 0.5, label = 'sample model')
        else:
            axs[0].plot(x2,func(x2,*s),'k-', zorder = 0, lw = 0.5, alpha = 1./np.sqrt(2**logN))
            # save prediction of sample model
        if predict:
            prediction.append(func(x2[ix],*s))
        
        # x-limits: choice to show prediction
    if not predict:
        axs[0].set_xlim([0,1])
    else:
        axs[0].set_xlim([0,3.2])
        ylim = axs[0].get_ylim(); axs[0].set_ylim(ylim)
        axs[0].plot([3,3],ylim, 'k--')
        
        # plot histogram of predictions
    if predict:
        h,e = np.histogram(prediction, bins = np.linspace(6,14,19))
        axs[1].bar(e[:-1], h, e[1]-e[0], color = [0.5,0.5,0.5])
        pcs = np.percentile(prediction, [5,95])
        ylim = axs[1].get_ylim(); axs[1].set_ylim(ylim)
        axs[1].plot([func(x2[ix],m0,c0), func(x2[ix],m0,c0)],ylim,'b-')
        axs[1].plot([pcs[0],pcs[0]],ylim,'k--')
        axs[1].plot([pcs[1],pcs[1]],ylim,'k--')
        
    # legend
    lns = ln1+ln2+ln3
    lbs = [ln.get_label() for ln in lns]
    axs[0].legend(lns,lbs,loc=2)
        
    # plot upkeep
    axs[0].set_xlabel('x',size = ts)
    axs[0].set_ylabel('y',size = ts)
    axs[1].set_xlabel('prediction at x=3',size = ts)
    axs[1].set_ylabel('frequency',size = ts)
    for ax in axs:
        for t in ax.get_xticklabels()+ax.get_yticklabels(): t.set_fontsize(ts)
            
    plt.show()
    
interact(plot_ensemble, logN = (0,10,1), predict = False)

## 3.5.4 Structural (or model) error

This refers to deficiencies in the construction of the model, which could include: the physics that are represented, discretization of the equations, assumptions made about homogeneity or features that are excluded.

In this example, we will consider structural error introduced by incorrect physics. When physical laws are used to formulate governing equations for a model, the result is a class of functions, $f(\cdot;\boldsymbol{\theta})$, whose parameters determine the behaviour of the model under particular circumstances. It is entirely possible to use the wrong parameters, $\boldsymbol{\theta}$, to try and predict the future: this is *parameter error*.

However, it is also possible that our interpretation of the relevant physics was incorrect and, actually, a different class of functions, $g(\cdot;\boldsymbol{\theta})$, provides a more appropriate description of the phenomenon we are attempting to model. We call this *structural (or model) error*.

In the examples presented here, we "know" the underlying model is linear in nature. However, let's assume that we mistakenly thought it was in fact logarithmic, i.e.,

$$y = a \ln(x_i-x_0)+b,$$

where $a$, $x_0$ and $b$ are the three parameters of this different model. Executing the code below will run through the same process in the cells above: a grid search is undertaken to find the objective function, and hence posterior, the posterior is sampled from to provide a forecast of the future. Compare how the two models' forecasts of the future differ and how they agree with the true outcome.

***Which model provides a better fit to the data?***

***How do the forecasts of the two models differ?***

In [ ]:

# vectors for making predictions
x2 = np.linspace(0.01,3.2, 101)
ix = np.argmin(abs(x2 - 3.0))

N = 2**10

# linear model samples
np.random.seed(13)
LNsamples = multivariate_normal.rvs(mean = [m1, c1], cov = cov, size = N)

# log model samples
    # define log model, three parameters
def func2(x,*p): 
    return p[0]*np.log(x+p[1])+p[2]
    # find best-fit log model
p2,pc = curve_fit(func2, xo, yo, [1,1,1])
    # construct parameter search grid
pi = np.linspace(p2[0]/10.,p2[0]*10.,51); dpi = pi[1]-pi[0]
pj = np.linspace(p2[1]/10.,p2[1]*10.,51); dpj = pj[1]-pj[0]
pk = np.linspace(p2[2]/10.,p2[2]*10.,51); dpk = pk[1]-pk[0]
PI, PJ, PK = np.meshgrid(pi,pj,pk)
# compute posterior for log model
    # empty vector, correct size, for storing computed objective function
r2 = 0.*PI.flatten() 
    # for each parameter combination in the grid search
for i,theta in enumerate(zip(PI.flatten(), PJ.flatten(), PK.flatten())):
        # compute objective function
    r2[i]=-np.sum((yo-func2(xo,*theta))**2)/v
    # reshape objective function to meshgrid dimensions
R2 = np.array(r2).reshape([len(pi), len(pj), len(pk)])
    # compute posterior
post2 = np.exp(R2/2.)
    # convert 2D mesh to vectors
pi,pj,pk,pv2 = [vi.flatten() for vi in [PI,PJ,PK,post2]]
# compute covariance matrix for log model
    # normalize posterior
pv2 = pv2/(np.sum(pv2))
    # means
pi1 = np.sum(pv2*pi)
pj1 = np.sum(pv2*pj)
pk1 = np.sum(pv2*pk)
    # variances
sii = np.sum(pv2*(pi-pi1)**2)
sjj = np.sum(pv2*(pj-pj1)**2)
skk = np.sum(pv2*(pk-pk1)**2)
sij = np.sum(pv2*(pi-pi1)*(pj-pj1))
sik = np.sum(pv2*(pi-pi1)*(pk-pk1))
sjk = np.sum(pv2*(pk-pk1)*(pj-pj1))
    # assemble matrix
cov2 = np.array([[sii,sij, sik],[sij, sjj, sjk],[sik, sjk, skk]])
    
np.random.seed(13)
LGsamples = multivariate_normal.rvs(mean = [pi1, pj1, pk1], cov = cov2, size = N)

# plotting function
def plot_structural(LNmodel, LNset, LGmodel, LGset):
        # initialize figure window and axes
    fig, axs = plt.subplots(nrows=1, ncols=2)
    fig.set_size_inches([16.,5.])
        # plot the observations
    axs[0].plot(xo,yo,'wo', mec = 'k', mew = 1.5, ms = 5, zorder = 10)
            
    # plot best-fit log model
    if LGmodel:
        axs[0].plot(x2,func2(x2,*p2),'g-',lw = 2)
        axs[0].plot(x2,func2(x2,*p2),'k--',lw = 2)
        
    # plot ensemble for best-fit log model
    if LGset:
        # plot line for each sample
        prediction = []
        for s in LGsamples: 
            axs[0].plot(x2, func2(x2,*s), 'g-', alpha = 1./np.sqrt(N))
            prediction.append(func2(x2[ix],*s))
        # plot histogram of predictions
        h,e = np.histogram(prediction, bins = np.linspace(4,14,33))
        axs[1].bar(e[:-1], h, e[1]-e[0], color = 'g', alpha = 0.5)
        pcs = np.percentile(prediction, [5,95])
        ylim = axs[1].get_ylim(); axs[1].set_ylim(ylim)
        axs[1].plot([func(x2[ix],m0,c0), func(x2[ix],m0,c0)],ylim,'b-')
        axs[1].plot([pcs[0],pcs[0]],ylim,'g--')
        axs[1].plot([pcs[1],pcs[1]],ylim,'g--')
    
    # plot best-fit linear model
    if LNmodel:   
        p1,pc = curve_fit(func, xo, yo, [1,1])
        axs[0].plot(x2,func(x2,*p1),'r-',lw = 2)
        axs[0].plot(x2,func(x2,*p1),'k--',lw = 2)
    
    # plot ensemble for linear model
    if LNset:
        # plot line for each sample
        prediction = []
        for s in LNsamples: 
            axs[0].plot(x2, func(x2,*s), 'r-', alpha = 1./np.sqrt(N))
            prediction.append(func(x2[ix],*s))
        # plot histogram of predictions
        h,e = np.histogram(prediction, bins = np.linspace(4,14,33))
        axs[1].bar(e[:-1], h, e[1]-e[0], color = 'r', alpha = 0.5)
        pcs = np.percentile(prediction, [5,95])
        ylim = axs[1].get_ylim(); axs[1].set_ylim(ylim)
        axs[1].plot([func(x2[ix],m0,c0), func(x2[ix],m0,c0)],ylim,'b-')
        axs[1].plot([pcs[0],pcs[0]],ylim,'r--')
        axs[1].plot([pcs[1],pcs[1]],ylim,'r--')
    
    # axis lims
    if LNset or LGset: 
        axs[0].set_xlim([0,3.2])
        ylim = axs[0].get_ylim()
    else: 
        axs[0].set_xlim([0,1])
        axs[0].set_ylim([1,6])
        
    # plot upkeep
    axs[0].set_xlabel('x',size = ts)
    axs[0].set_ylabel('y',size = ts)
    axs[1].set_xlabel('prediction at x=3',size = ts)
    axs[1].set_ylabel('frequency',size = ts)
    for ax in axs:
        for t in ax.get_xticklabels()+ax.get_yticklabels(): t.set_fontsize(ts)
    plt.show()
    
interact(plot_structural, LNmodel=False, LNset=False, LGmodel=False, LGset=False)